# 02 Problem Solving Paradigms
## 02B Dynamic Programming

## Foreword

Dynamic Programming (henceforth abbreviated as DP) is perhaps the most challenging problem solving paradigm discussed in this section. Thus, *ensure you have mastered the techniques in Module 02A* and prepare for **a lot** of recursion and recurrance relations!

The key skills you will need to develop in order to master DP is:
1. The ability to identify and determine problem *states*
2. The ability to determine the relationship(s) between the current problem and its subproblems (i.e. identify the *transition*)

In fact, DP is often called 'recursion to the max' - it is a more efficient implementation of recursive techniques. While you are starting out with DP, you *should* use Top-Down DP (this will be explained later) and understand it as a sort-of 'intelligent' recursion.

DP is most often used to solve **optimisation problems** and **counting problems**. If you encounter a problem that says "maximise this" or "minimise that" or "count the number of ways that so and so can happen", it is *highly likely* that it is a DP problem.

Most DP problems in contests ask for the optimal/Total *value* and not the optimal *solution* as this removes the need to recursively backtrack to find the actual solution. Harder DP problems, however, *may* ask for the optimal state.

## Precursor: Recurrance Relations

Recurrance relations is a fancy mathematical term for "how recurrance operations are related to each other".

Consider the Fibbonacci sequence:
$$a_{n+2} = a_{n+1} + a_n$$
with $a_1 = a_2 = 1$. This is an example of a recurrance relation, albeit a fairly *boring* one.

Recurrance relations are useful in obtaining the next state of the DP solution. We cannot go into much detail in this section, but it should be said that it is *much more complex* than the relation shown above.

To practice recurrance relations, try out this problem (labelled problem 0 in the problem set):
0. Domino Tiling

## Conditions for DP to be Applied

Just like the Greedy algorithm, a problem has to satisfy **two** conditions in order for a DP solution to work.
1. It has optimal sub-structures.<br>The solution to a subproblem is part of the solution to the main problem.
2. It has overlapping subproblems.<br>This is the **key characteristic** of DP! What this essentially means is that **we will need to access previous subproblems' solutions repeatedly** in the process of solving the main problem.

Now for DP we are interested in the number of **distinct** subproblems that we have to solve. This is the **number of DP _states_** as mentioned in the Foreword. Finding the way to relate these states with one another is finding the **transition**.

## The Classic Example: Coin Change

Problem statement: Given a set of coins $S$ and a target amount $V$, find the minimum number of coins that are needed to form $V$. It is guarrenteed that it is possible to form $V$ using the coins in $S$. Assume we have an unlimited supply of coins, and a coin valued at 1 is always present in $S$.

Earlier we looked at the Greedy solution to this problem, but it does not work for all cases. We gave the example of $S = [1, 7, 10]$ and $V = 14$ to show that the Greedy solution was not optimal. **The DP solution, however, will guarrentee optimality.**

Let's look at what we want to achieve in the end - minimum number of coins needed to achieve a certain value $V$. We note three things.
1. To achieve a value $x$ we will never need to use a coin that valued more than $x$.
    - This is quite obvious.
2. To achieve a value $v$ we must come from a value $s$ where $s < v$.
    - Again, this is quite obvious. It is simply impossible to achieve the value $v$ by coming from a higher value; there are no negative-valued coins!
3. Optimal solution for achieving value $v$ means we have achieved optimal solution for value $s$, where $s < v$.
    - This is less obvious. Suppose we do not achieve optimality in $s$. By point 2 we know we must have reached value $v$ from value $s$. However if $s$ is non-optimal there is **no way** that $v$ is optimal!

Taking these three things together means that we have optimal substructure **and** overlapping subproblems - perfect for applying DP on!

Now to analyse the problem's transition. *How do we get from a value $s$ to a value $v$?* Well, we choose one of the coins' values that are in $S$ and add it to $s$ to achieve a new value.

Let's do this in reverse - to reach $v$ we must have added one of the coins inside the set $S$ from a smaller value. All we have to do is **consider every single possibility of a coin value from $S$** (say the coin value is $c$) and find the optimal solution to $v - c$.

Now we know **one base case** for our recursion: making $0$ dollars takes 0 coins. With this base case, we are ready to write the **recursive solution**.

In [1]:
# IMPORTS
from math import inf  # Gets the placeholder value of infinity


# FUNCTIONS
def coin_change(value, possible_coins):
    # Base case: value = 0 takes 0 coins
    if value == 0:
        return 0

    # Otherwise, consider every possible coin and calculate the optimal solution from there
    min_coins = inf

    for coin in possible_coins:
        # The value that we consider can't be negative, so check if subtracting that value of coin makes it
        # too small
        curr_val = value - coin
        if curr_val >= 0:
            # Get the optimal solution
            min_coins = min(min_coins, coin_change(curr_val, possible_coins) + 1)  # Add this coin

    # Return the minimum number of coins
    return min_coins


# MAIN CODE
coins = [1, 7, 10]
target = 14
print(coin_change(target, coins))

2


Note that this is **still not a DP solution**. There are a lot of redundancy in recalculating cases if we use pure recursion. To make it from a *recursive solution* into a *Top-Down DP solution*, we need to use ***memoisation***.

Memoisation is an optimization technique used primarily to speed up computer programs by **storing the results of function calls** and **returning the cached result** when the same inputs occur again. This reduces the number of computations.

With memoisation, we can implement a Top-Down DP solution. It is called "Top-Down" as we are starting with the *final state* (that is, the target value we want to find the optimal solution for) and working our way through the subproblems.

A Top-Down DP solution for the coin change problem is provided below. Note the addition of a *global* dictionary `memo`, which stores the computed function calls.

In [2]:
# DP FUNCTION
def coin_change(target, coin_values, memo_dict):  # Note the new parameter
    # 1. Base case(s)
    if target == 0:
        return 0  # We need 0 coins to form the value of 0

    # 2. Check if we processed that state already
    key = target
    if key in memo_dict:  # We've computed this already
        return memo_dict[key]

    # 3. Process all possible cases
    all_cases = []
    for coin in coin_values:
        curr_val = target - coin
        if curr_val >= 0:
            all_cases.append(coin_change(curr_val, coin_values, memo_dict) + 1)

    # 4. Get the optimal solution
    answer = min(all_cases)

    # 5. Update the memoisation dictionary
    memo_dict[key] = answer  # Using the same key as above

    # 6. Return answer
    return answer


# MAIN CODE
coins = [1, 7, 10]
target = 14

memo = {}  # Dictionary to store recursive function calls
print(coin_change(target, coins, memo))

2


We can now print the `memo` dictionary to see the computed values.

In [3]:
print(memo)

{1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 1, 8: 2, 9: 3, 10: 1, 11: 2, 12: 3, 13: 4, 14: 2}


Now the Top-Down DP is not the 'true form' of DP. The true form of DP is actually known as **tabulation**, or **Bottom-Up DP**.

## Bottom-Up DP

Unlike the Top-Down DP solution which **requests the answers of subproblems as the algorithm requires them**, Bottom-Up DP **generates all possible states of the problem, starting from the base case**. This makes it more efficient than a Top-Down DP solution, as it is pure iteration rather than needing recursive calls.

The basic steps to construct a Bottom-Up DP solution are as follows.
1. Determine the set of *parameters* that **uniquely describe the problem (i.e. the _state_)**. This is similar to defining the `key` in the Top-Down DP solution above - the `key` uniquely describes each state.
2. If $N$ parameters are needed to describe the state, prepare an **$N$ dimensional array** representing the DP table, with one entry per state. This is similar to the `memo` dictionary in the above example, with one key difference: the `memo` dictionary is empty, but the DP array should be **initialised with the base case values**.
3. Now with the base case cells filled, determine the states in the DP table that can be filled next (the transition). Repeat until the entire DP table is filled. For Bottom-Up DP, this is accomplished through the use of **loops**.

### Coin Change With Bottom-Up DP

To solve the coin change problem more efficiently, we can consider a Bottom-Up DP solution in place of a Top-Down DP approach.

Since the only parameter that uniquely describes each state of the problem is the **target value**, we can define a 1-dimension DP table, where the index $i$ (0-based indexing) represents the **target value**.

The Bottom-Up DP solution is quite similar to the Top-Down DP solution.

In [4]:
# IMPORTS
from math import inf

# DP FUNCTION
def coin_change(target, coin_values):
    # Define the DP array
    dp = [None] * (target + 1)  # Add 1 so that we can access the target's index

    # Set base cases
    dp[0] = 0  # It takes 0 coins to form the value of 0

    # Iterate through the states
    for val in range(1, target + 1):
        # Process all possible cases
        min_coins = inf

        for coin in coin_values:
            # Check if the previous state to consider is valid
            prev_state = val - coin
            if prev_state >= 0:
                # Update the number of coins needed, as necessary
                min_coins = min(min_coins, dp[prev_state] + 1)

        # Update the current state in the DP array
        dp[val] = min_coins

    # Return the required state
    print(dp)  # For debugging
    return dp[target]


# MAIN CODE
coins = [1, 7, 10]
target = 14
print(coin_change(target, coins))

[0, 1, 2, 3, 4, 5, 6, 1, 2, 3, 1, 2, 3, 4, 2]
2


## Comparing DP Styles

For *most* DP problems, the two styles of DP (that is, Bottom-Up DP and Top-Down DP) are equally good and the decision to use a particular DP style is a matter of preference. However, for *harder* DP problems, one of the styles can be better than the other.

The following is a list of pros and cons of each style of DP.

|      | **Top-Down DP**                                                                                                                                                                                                                                                                              | **Bottom-Up DP**                                                                                                                                                                          |
|------|:---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|:------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| Pros | 1. It is a natural transformation from a recursion solution<br><br>2. Completes the subproblems only when necessary (sometimes this is better)                                                                                                                                               | 1. Faster if many sub-problems are revisited as there is no overhead from recursive calls<br><br>2. Can save memory space                                                                 |
| Cons | 1. Slower if many subproblems are revisited due to function call overhead <br>   <br>   (but this is not usually penalised in competitions)<br><br>2. If there are $M$ states, an $O(M)$ DP memoisation dictionary is needed, <br>   <br>   which may lead to `MLE` for some harder problems | 1. For programmers who are inclined to recursion, this style may not be intuitive<br><br>2. If there are $M$ states, Bottom-Up DP will visit and fill in the value of all of these states |

## Another Classic Example: The 0-1 Knapsack Problem

Here's another classic DP problem, known as the 0-1 knapsack problem. Given $n$ items, each with its own weight $W_i$ and value $V_i$, and a knapsack that can only hold up to $S$ in weight, what is the maximum value of the items inside the knapsack? Assume that we can only either take the item or ignore the item (hence the term 0-1 for either ignoring or taking).

For example, $n=4$, $S=12$, $V = [100,70,50,10]$ and $W = [10,4,6,2]$.
- If we select item 0 with weight 10 and value 100, we cannot take any other item. Not optimal.
- If we select item 3 with weight 12 and value 10, we cannot take any other item. Not optimal.
- If we select item 1 and 2, we have total weight 10 and total value 120. This is the maximum.

We can formulate the following recurrances regarding `val(id, remW)` which returns the optimal value, where `id` is the current item's index and `remW` is the remaining weight in the knapsack.
1. `val(id, 0) = 0`. This is because if the remaining weight is 0, we cannot take anything else. Thus the optimal value is 0.
2. `val(n, remW) = 0`. This is because the current item index is $n$, which means we have already considered all items. Thus the optimal value is 0.
3. If `W[id] > remW`, then `val(id, remW) = val(id + 1, remW)`. This is because the current item is too heavy for us to consider. Thus we ignore the current item and move on to the next item.
4. If `W[id] <= remW`, we have two choices: ignore the item or take the item.
    - If we ignore the item the value we get is `val(id + 1, remW)` (since we are moving on to the next item)
    - If we take the item the value we get is `V[id] + val(id + 1, remW - W[id])`. This is because taking the item adds that item's value to the current tally, but reduces the remaining weight available to us.
   
   Thus `val(id, remW)` is the **maximum** of these two values, i.e. `val(id, remW) = max(val(id + 1, remW), V[id] + val(id + 1, remW - W[id]))`.

The answer to the problem can be found by calling `val(0, S)`.

Note the overlapping subproblems in the 0-1 knapsack problem. For example, after **taking** item 0 and **ignoring** item 1 and 2, we arrive at state `(3, 2)` (at the 3rd item (0-based indexing) and with 2 units of weight remaining). After **ignoring** item 0 and **taking** item 1 and 2, we arrive at the **same state** of `(3, 2)`. Although there are overlapping subproblems, there are only $O(nS)$ distinct states. We can compute each of these states in $O(1)$ time, and thus the overall time complexity of this solution is $O(nS)$.

The Python implementation of the above algorithm is provided below, under the function name `knapsack`.

In [5]:
# DP FUNCTION
def knapsack(item_index, remaining_weight, weights, values, memo_dict):
    # 0. Define needed variables
    n = len(weights)  # Number of items

    # 1. Base Cases
    if remaining_weight == 0:  # If we have no more weight to spare, we cannot take anything
        return 0               # Hence the value of items in our basket is 0
    
    if item_index == n:  # We've reached the end of the list of items; we cannot consider any more items
        return 0         # Hence the value of items in our basket is 0

    if weights[item_index] > remaining_weight:  # The current item's weight is too much
        # Consider the next item
        return knapsack(item_index + 1, remaining_weight, weights, values, memo_dict)

    # 2. Check if we already processed the state already
    key = f"{item_index} {remaining_weight}"  # This uniquely defines this state
    
    if key in memo_dict:  # If already processed
        return memo_dict[key]

    # 3. Process all possible cases
    # Option 1: We take this item
    take_item_value = values[item_index] + knapsack(item_index + 1, remaining_weight - weights[item_index], 
                                                    weights, values, memo_dict)

    # Option 2: We ignore this item
    ignore_item_value = knapsack(item_index + 1, remaining_weight, weights, values, memo_dict)

    # 4. Get optimal solution (Optimal solution achieved when value is maximal)
    max_val = max(take_item_value, ignore_item_value)

    # 5. Update the memoisation dictionary
    memo[key] = max_val

    # 6. Return the answer
    return max_val


# MAIN CODE
values = [100, 70, 50, 10]
weights = [10, 4, 6, 2]
maxWeight = 12

memo = {}  # Dictionary to store recursive function calls
print(knapsack(0, maxWeight, weights, values, memo))

130


We can now print the `memo` dictionary to see the computed values. Remember that, in the key, the first integer is the **item index** and the second integer is the **remaining weight**.

In [6]:
print(memo)

{'3 2': 10, '3 8': 10, '2 8': 60, '3 6': 10, '3 12': 10, '2 12': 60, '1 12': 130, '0 12': 130}


*Note*: The top-down version of this DP solution **is often faster** than the bottom-up version. This is because **not all states are actually visited** adnt hence the cirtical DP states involved are only a (vert small) subset of the entire state space. *Remember: top-down DP only visits __required states__ whereas bottom-up DP visits __all distinct states__*.

## Displaying the Optimal Solution

Many DP problems require only the value of the optimal solution (such as the problems above). However many are caught off-guard when they are asked to **print the optimal solution**. We are aware of two ways of doing so.

The first way is mainly used in Bottom-Up DP solutions (but is still applicable to Top-Down DP solutions) where we store the predecessor information at every state. If there are more than 1 optimal predecessor and the question asks us to print **all** of the optimal solutions, we can store the optimal solutions in a list at every state. Once we have the optimal final state, we can do backtracking to recreate the optimal solution path.

The second way is mainly applicable to the Top-Down DP approach where we utilise recursion and backtracking to recreate the optimal solution. This can be done by making an *auxiliary function* that has the **same structure** as the original DP function but uses the `memo` table to reconstruct the optimal solution.

Reconstruction of the optimal solution for the **Coin Change** problem (using Bottom-Up DP solution) is given below.

In [7]:
# IMPORTS
from math import inf

# DP FUNCTION
def coin_change_with_solution(target, coin_values):
    # Define the DP array
    dp = [None] * (target + 1)  # Add 1 so that we can access the target's index

    # Set base cases
    dp[0] = (0, 0)  # It takes 0 coins to form the value of 0, and this has no predecessor

    # Iterate through the states
    for val in range(1, target + 1):
        # Process all possible cases
        min_coins = inf
        best_state = None

        for coin in coin_values:
            # Check if the previous state to consider is valid
            prev_state = val - coin
            if prev_state >= 0:
                # Update the number of coins needed, as necessary
                num_coins_needed = dp[prev_state][0] + 1  # Take the number of coins needed, not previous state
                if min_coins > num_coins_needed:
                    min_coins = num_coins_needed
                    best_state = prev_state

        # Update the current state in the DP array
        dp[val] = (min_coins, best_state)  # Difference in current value and best state gives the coin used

    # Generate the optimal solution
    curr_index = target
    optimal_solution = []

    while curr_index != 0:
        prev_index = dp[curr_index][1]  # Get the previous state's index
        optimal_solution.append(curr_index - prev_index)  # This gives the coin used
        curr_index = prev_index

    # Return the optimal solution
    print(dp)  # For debugging
    return optimal_solution


# MAIN CODE
coins = [1, 7, 10]
target = 14
print(coin_change_with_solution(target, coins))

[(0, 0), (1, 0), (2, 1), (3, 2), (4, 3), (5, 4), (6, 5), (1, 0), (2, 7), (3, 8), (1, 0), (2, 10), (3, 11), (4, 12), (2, 7)]
[7, 7]


Reconstruction of the optimal solution for the **0-1 Knapsack** Problem (using Top-Down DP solution) is given below.

In [8]:
# DP FUNCTION
def knapsack(item_index, remaining_weight, weights, values, memo_dict):
    # 0. Define needed variables
    n = len(weights)  # Number of items

    # 1. Base Cases
    if remaining_weight == 0:  # If we have no more weight to spare, we cannot take anything
        return 0               # Hence the value of items in our basket is 0
    
    if item_index == n:  # We've reached the end of the list of items; we cannot consider any more items
        return 0         # Hence the value of items in our basket is 0

    if weights[item_index] > remaining_weight:  # The current item's weight is too much
        # Consider the next item
        return knapsack(item_index + 1, remaining_weight, weights, values, memo_dict)

    # 2. Check if we already processed the state already
    key = f"{item_index} {remaining_weight}"  # This uniquely defines this state
    
    if key in memo_dict:  # If already processed
        return memo_dict[key]

    # 3. Process all possible cases
    # Option 1: We take this item
    take_item_value = values[item_index] + knapsack(item_index + 1, remaining_weight - weights[item_index], 
                                                    weights, values, memo_dict)

    # Option 2: We ignore this item
    ignore_item_value = knapsack(item_index + 1, remaining_weight, weights, values, memo_dict)

    # 4. Get optimal solution (Optimal solution achieved when value is maximal)
    max_val = max(take_item_value, ignore_item_value)

    # 5. Update the memoisation dictionary
    memo[key] = max_val

    # 6. Return the answer
    return max_val


def get_knapsack_solution(item_index, remaining_weight, weights, values, memo_dict, taken_items=[]):
    # Define needed variables
    n = len(weights)  # Number of items

    # Base Cases
    if remaining_weight == 0:  # If we have no more weight to spare, we cannot take anything
        return taken_items     # Hence return the items that we have already taken
    
    if item_index == n:     # We've reached the end of the list of items; we cannot consider any more items
        return taken_items  # Hence return the items that we have already taken

    if weights[item_index] > remaining_weight:  # The current item's weight is too much
        # Consider the next item
        return get_knapsack_solution(item_index + 1, remaining_weight, weights, values, memo_dict, taken_items)
    
    # Check if the option to take is the optimal solution
    key = f"{item_index} {remaining_weight}"  # This uniquely defines this state
    take_item_value = values[item_index] + knapsack(item_index + 1, remaining_weight - weights[item_index], 
                                                    weights, values, memo_dict)
    
    if take_item_value == memo_dict[key]:
        # We've taken the item; append to the list of taken items
        taken_items.append(item_index)

        # Proceed onto the next case
        return get_knapsack_solution(item_index + 1, remaining_weight - weights[item_index], weights, values, 
                                     memo_dict, taken_items)
    
    # So we didn't take the item; move on to the next item then
    return get_knapsack_solution(item_index + 1, remaining_weight, weights, values, memo_dict, taken_items)


# MAIN CODE
values = [100, 70, 50, 10]
weights = [10, 4, 6, 2]
maxWeight = 12

memo = {}  # Dictionary to store recursive function calls
knapsack(0, maxWeight, weights, values, memo)  # Must run the Top-Down DP at least once
print(get_knapsack_solution(0, maxWeight, weights, values, memo))  # Now we can generate the solution

[1, 2, 3]


However it should be noted that it is rare for a question to ask for the optimal solution. It is far more common for the question to only ask for the **value** of the optimal solution.

## Problems

This is a long module, but there are not many problems associated with this module. *However, the problems are of higher difficulty* and may need more thinking and analysing to solve them. **This is expected of DP problems**. It is the hardest of the four problem-solving paradigms, after all.

0. Domino Tiling
1. Maximal Rising Subsequence
2. Bricks Game
3. Knapsack
4. Jill Rides Again